In [1]:
#Read the chunked data
import pandas as pd
df=pd.read_csv("Chunked_data.csv")
df

,Unnamed: 0,name,Chunked Data
0,0,the message (1976),watch any video online with opensubtitles free...
1,1,the message (1976),sts brought their idols and the images of thei...
2,2,the message (1976),he is 40 years old its unnatural with a rich w...
3,3,the message (1976),y in the shape of a man standing on the horizo...
4,4,the message (1976),but whilst im alive you will obey your father...
...,...,...,...
427628,427628,scorpion s04 e20 foul balls (2018),t thing that you guys have no romantic feeling...
427629,427629,scorpion s04 e20 foul balls (2018),ey where are you going well looks like kyles g...
427630,427630,scorpion s04 e20 foul balls (2018),e all wearing the same shirts it shows were a ...
427631,427631,scorpion s04 e20 foul balls (2018),at the snot out of it homeland player we got t...


In [2]:
copy_df=df.copy()
copy_df.head()

,Unnamed: 0,name,Chunked Data
0,0,the message (1976),watch any video online with opensubtitles free...
1,1,the message (1976),sts brought their idols and the images of thei...
2,2,the message (1976),he is 40 years old its unnatural with a rich w...
3,3,the message (1976),y in the shape of a man standing on the horizo...
4,4,the message (1976),but whilst im alive you will obey your father...


## Interaction with ChromaDB

In [3]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH="chroma_subtitle_data/" #path where the chromadb data is present
EMBED_MODEL="all-MiniLM-L6-v2"  #bert based embedding model
COLLECTION_NAME="subtitle_movie_data" #collection name
#client connection
client=chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [4]:
embedding_func=embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)
collection=client.create_collection(name=COLLECTION_NAME,embedding_function=embedding_func,metadata={"hnsw:space":"cosine"},)

## Adding the data to the collection

In [5]:
import pandas as pd
from chromadb import Collection

In [6]:
chunked_data=copy_df['Chunked Data'].tolist()

In [7]:
chunked_data

['watch any video online with opensubtitles free browser extension osdblinkext in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patriarch of alexandria kisra emperor of persia muhammad calls you with the call of god accept islam for your salvation embrace islam you come out of the desert smelling of camel and goat to tell persia where he should kneel muhammad messenger of god who gave him this authority god sent muhammad as a mercy to mankind the scholars and historians of islam  the university of alazhar in cairo the high islamic congress of the shiat in lebanon the makers of this film honour the isl

In [10]:
names=copy_df['name'].tolist()
names

['the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'the message (1976)',
 'here comes the grump s01 e09 joltin jack in boxia (1969)',
 'here comes the grump s01 e09 joltin jack in boxia (1969)',
 'here comes the grump s01 e09 joltin jack in boxia (1969)',
 'yumis cells s02 e13 episode 2 13 (2022)',
 'yumis cells s02 e13 episode 2 13 (2022)',
 'yumis cells s0

In [13]:
max_batch_size = 41660
num_batches = -(-len(chunked_data) // max_batch_size)

#function to add data batchwise
for i in range(num_batches):
    start_index = i * max_batch_size
    end_index = min((i + 1) * max_batch_size, len(chunked_data))
    batch_data = chunked_data[start_index:end_index]
    batch_names = names[start_index:end_index]

    batch_ids = [f"id{j}" for j in range(start_index, end_index)]

    batch_metadata = [{"names": n} for n in batch_names]

    # Adding the data batch-wise
    collection.add(
        documents=batch_data,
        ids=batch_ids,
        metadatas=batch_metadata
    )